In [7]:
import csv
import numpy as np
import pandas as pd
DS = pd.read_csv("MergedDSFULL.csv")

C:\Users\Kasutaja\Anaconda3\envs\myEnv\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
DS.head()

,text_ID,start,end,object,value,unit,min,max,text
0,46809,0,6,RR,130,NaN,NaN,NaN,RR 130/80 mmHg.
1,43588,2421,2433,ALP,54,U/L,NaN,NaN,UURINGUD JA PROTSEDUURID \nLÄBIVAATUS: Objekt...
2,43050,720,735,HCT,40,%,NaN,41.2,UURINGUD JA PROTSEDUURID \n|Uuring|24.04|25.0...
3,45389,165,171,RR,110,NaN,NaN,NaN,"Kergelt liigkaaluline. Nahk, limaskestad taval..."
4,42960,2314,2332,HCT,45,%,40.0,50.0,UURINGUD JA PROTSEDUURID \nObjektiivselt: Nor...


In [17]:
# DS150 = DS[DS['text'].apply(len)<150]
DS150 = DS[DS['text'].apply(lambda x: len(str(x)) < 150)]

In [22]:
DSIds = set(DS150.text_ID)

In [30]:
DSIDSList = list(DSIds)

In [32]:
DSIDSList.sort()

In [39]:
DSIDSList[12999]

304534

In [40]:
DS150Subset = DS150[DS150['text_ID'] <= 304534]

In [41]:
len(DS150Subset)

33849

In [42]:
len(set(DS150Subset["text_ID"]))

13000

In [44]:
len(set(DS150Subset["object"]))

176

In [51]:
ObjectStats = DS["object"].value_counts()

In [52]:
ObjectStats

RR                                           138495
MCV                                           54799
MCH                                           52742
MCHC                                          52696
fr                                            51654
                                              ...  
juurde tulnud - talvega                           1
kaalulangus 10 kg viimase 2 kuu jooksul           1
juurde võtnud umbes                               1
kaalus alla. RR kodus ja tööl 124-138/85-         1
langenud - seega                                  1
Name: object, Length: 2125, dtype: int64

In [54]:
ObjectStats.describe()

count      2125.000000
mean        768.403294
std        5068.159364
min           1.000000
25%           1.000000
50%           2.000000
75%          15.000000
max      138495.000000
Name: object, dtype: float64

In [55]:
DS150Subset.to_csv("DS150SubsetAllObjects.csv",index=False)

In [56]:
DS150Subset.head()

,text_ID,start,end,object,value,unit,min,max,text
0,46809,0,6,RR,130,NaN,NaN,NaN,RR 130/80 mmHg.
5,47482,0,5,RR,142,NaN,NaN,NaN,"RR142/89mmHg, HR 76 x min, p167cm, k89kg. EKG..."
6,53393,0,5,RR,143,NaN,NaN,NaN,"RR143/87mmHg, HR60 x min, p180cm, k93kg"
8,51616,98,103,fr,72,NaN,NaN,NaN,\nNahaleid: puhas\nHingamiselundite leid: vesi...
26,101498,15,23,fr,83,x',NaN,NaN,"Cor regul rütm fr 83 x', toonid ii, RR 150/100..."


In [79]:
DS = DS150Subset.copy()
new_df = pd.DataFrame(DS["text"].str.split().tolist(), index=DS.text_ID).stack()
new_df = new_df.reset_index([0, 'text_ID'])
new_df.columns = ['text_ID', 'word']
new_df["tag"]= np.NaN
myList= list(new_df.text_ID.unique())
for i in range(len(myList)):
    Original = DS.loc[DS["text_ID"] == myList[i]]
    words = new_df.loc[new_df["text_ID"] == myList[i]]
    for OriginalIndex, OriginalRow in Original.iterrows():
        for index, row in words.iterrows():
                if(str(Original["object"][OriginalIndex]) in str(words["word"][index])):
                    new_df["tag"][index] = "object"
                elif(not float(Original["value"][OriginalIndex]).is_integer()):
                    if(str(Original["value"][OriginalIndex]) in str(words["word"][index])):
                        new_df["tag"][index] = "object"
                elif(str(int(Original["value"][OriginalIndex])) in str(words["word"][index])):
                    new_df["tag"][index] = "object"


In [76]:
new_df = new_df[0:319755]

In [77]:
len(set(new_df.text_ID))

12100

In [78]:
new_df.to_csv("DS150SubsetAllObjectsPrepared.csv",index=False)

In [8]:
len(new_df)

157280

In [9]:
len(new_df.text_ID.unique())

13643

In [80]:
new_df["tag"].value_counts()

object    9955
Name: tag, dtype: int64

In [81]:
new_df["tag"].isna().sum()

309800

In [82]:
new_df.head()

,text_ID,word,tag
0,46809,RR,object
1,46809,130/80,object
2,46809,mmHg.,NaN
3,47482,"RR142/89mmHg,",object
4,47482,HR,object


In [83]:
words = list(set(new_df["word"].values))
n_words = len(words);

In [84]:
n_words

17330